In [1]:
%pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import linear_model
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

dataset = datasets.load_diabetes(as_frame = True)
X,y = dataset.data, dataset.target
#X = (X - X.mean()) / X.std()
#X.drop(columns=['s5', 'bmi'], inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)
reg = linear_model.LinearRegression()
model_lr = reg.fit(X=X_train, y=y_train)
mse_lr = mean_squared_error( y_test, model_lr.predict(X_test) )
r2_lr = r2_score(y_test, model_lr.predict(X_test))
print( mse_lr )
print("r2_score = %s" % r2_lr )
print( model_lr.coef_ )


3131.1146628350402
r2_score = 0.4429203769660095
[  72.47188595 -223.21161165  657.48915455  314.24795529 -562.76315416
  331.1350922   -83.0521782   -75.50236282  816.50308987  -86.88439733]


In [4]:
X['res']=y
X.corr()
X.drop(columns='res')

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [5]:
dataset = datasets.load_diabetes(as_frame = True)
X,y = dataset.data.copy(), dataset.target.copy()
X.loc[X['sex'] > 0, 'sex'] = 1
X.loc[X['sex'] < 0, 'sex'] = 0
X.loc['sex'] = X['sex'].astype('int')

In [6]:
from catboost import CatBoostRegressor
dataset = datasets.load_diabetes(as_frame = True)
X,y = dataset.data.copy(), dataset.target.copy() # for some reason I should use copy() to get rid of warnings

X.loc[X['sex'] > 0, 'sex'] = 1
X.loc[X['sex'] < 0, 'sex'] = 0
X['sex'] = X['sex'].astype('int')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

#X_train['sex', X_train['sex'] <= 0] = 0
#X_train['sex'].astype('int64')
model_cb = CatBoostRegressor(iterations=4000,  # Количество итераций
                             learning_rate=0.001,  # Скорость обучения
                             depth=3,  # Глубина дерева
                             cat_features=[1],  # Индексы категориальных признаков
                             random_state=0)  # Задаем случайное начальное состояние для воспроизводимости
model_cb.fit(X_train, y_train, verbose=10000)
mse_cb = mean_squared_error( y_test, model_cb.predict(X_test) )
r2_cb = r2_score(y_test, model_cb.predict(X_test))
print( mse_cb )
print("r2_score = %s" % r2_cb )

0:	learn: 81.5119991	total: 53.1ms	remaining: 3m 32s
3999:	learn: 44.9288469	total: 1.78s	remaining: 0us
3155.5527670423476
r2_score = 0.43857241422894844


In [7]:
import lightgbm as lgb

dataset = datasets.load_diabetes(as_frame = True)
X,y = dataset.data.copy(), dataset.target.copy()
X['sex'][X['sex'] > 0] = 1
X['sex'][X['sex'] < 0] = 0
X['sex'] = X['sex'].astype('category')
X.rename(columns={'sex':'cat_feature'})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

# this shit doesn't fucking work
#lgX_train = lgb.Dataset(data=X_train,
#                        feature_name=X_train.columns.tolist(),
#                        categorical_feature=['sex'])
#lgX_test = lgb.Dataset(data=X_test,
#                       feature_name=X_test.columns.tolist(),
#                       categorical_feature=['sex'])

model_lgb = lgb.LGBMRegressor(num_leaves=5,  # Количество листьев в дереве
                              learning_rate=0.01,  # Скорость обучения
                              n_estimators=164,  # Количество деревьев
#                          categorical_feature=1, because this shit doesn't work
                              random_state=0)  # Задаем случайное начальное состояние для воспроизводимости
model_lgb.fit(X_train, y_train)
mse_lgb = mean_squared_error( y_test, model_lgb.predict(X_test) )
r2_lgb = r2_score(y_test, model_lgb.predict(X_test))
# model.fit(X_train, y_train, verbose_eval=1, callbacks=[lgb.log_evaluation()])
# because verbose and callbacks don't fucking work

print(f"Mean Squared Error: {mse_lgb}")

# catboost works, this shit does not

Mean Squared Error: 3388.2908612250794


In [8]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
dataset = datasets.load_diabetes(as_frame = True)
X,y = dataset.data.copy(), dataset.target.copy() # for some reason I should use copy() to get rid of warnings

X.loc[X['sex'] > 0, 'sex'] = 1
X.loc[X['sex'] < 0, 'sex'] = 0
X['sex'] = X['sex'].astype('int')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

#X_train['sex', X_train['sex'] <= 0] = 0
#X_train['sex'].astype('int64')
model = CatBoostRegressor(cat_features=[1], verbose=False)

# Определение гиперпараметров и их значений для подбора
param_grid = {
    'learning_rate': [0.008, 0.009, 0.01, 0.011, 0.012],
    'depth': [1, 2, 3],
    'iterations': [980, 990, 1000, 1010, 1020]
}

# Подбор гиперпараметров с использованием GridSearchCV
model_gs = GridSearchCV(model, param_grid, cv=4, verbose=3, scoring='neg_mean_squared_error')
model_gs.fit(X_train, y_train)
mse_gs = mean_squared_error( y_test, model_gs.predict(X_test) )
r2_gs = r2_score(y_test, model_gs.predict(X_test))
# Вывод наилучших параметров и значения метрик
print("Наилучшие параметры: ", model_gs.best_params_)
print("Наилучшее значение RMSE на тестовом наборе: ", np.sqrt(-model_gs.score(X_test, y_test)))

print( mse_gs )
print("r2_score = %s" % r2_gs )

Fitting 4 folds for each of 75 candidates, totalling 300 fits
[CV 1/4] END depth=1, iterations=980, learning_rate=0.008;, score=-3258.327 total time=   0.3s
[CV 2/4] END depth=1, iterations=980, learning_rate=0.008;, score=-1918.822 total time=   0.2s
[CV 3/4] END depth=1, iterations=980, learning_rate=0.008;, score=-2915.477 total time=   0.2s
[CV 4/4] END depth=1, iterations=980, learning_rate=0.008;, score=-3846.118 total time=   0.2s
[CV 1/4] END depth=1, iterations=980, learning_rate=0.009;, score=-3191.968 total time=   0.3s
[CV 2/4] END depth=1, iterations=980, learning_rate=0.009;, score=-1929.584 total time=   0.2s
[CV 3/4] END depth=1, iterations=980, learning_rate=0.009;, score=-2901.125 total time=   0.4s
[CV 4/4] END depth=1, iterations=980, learning_rate=0.009;, score=-3836.416 total time=   0.2s
[CV 1/4] END depth=1, iterations=980, learning_rate=0.01;, score=-3135.486 total time=   0.2s
[CV 2/4] END depth=1, iterations=980, learning_rate=0.01;, score=-1932.119 total tim

[CV 3/4] END depth=1, iterations=1020, learning_rate=0.009;, score=-2902.577 total time=   0.2s
[CV 4/4] END depth=1, iterations=1020, learning_rate=0.009;, score=-3835.400 total time=   0.2s
[CV 1/4] END depth=1, iterations=1020, learning_rate=0.01;, score=-3118.510 total time=   0.2s
[CV 2/4] END depth=1, iterations=1020, learning_rate=0.01;, score=-1937.026 total time=   0.2s
[CV 3/4] END depth=1, iterations=1020, learning_rate=0.01;, score=-2918.746 total time=   0.2s
[CV 4/4] END depth=1, iterations=1020, learning_rate=0.01;, score=-3822.473 total time=   0.2s
[CV 1/4] END depth=1, iterations=1020, learning_rate=0.011;, score=-3081.372 total time=   0.2s
[CV 2/4] END depth=1, iterations=1020, learning_rate=0.011;, score=-1955.914 total time=   0.3s
[CV 3/4] END depth=1, iterations=1020, learning_rate=0.011;, score=-2929.894 total time=   0.2s
[CV 4/4] END depth=1, iterations=1020, learning_rate=0.011;, score=-3809.531 total time=   0.2s
[CV 1/4] END depth=1, iterations=1020, learn

[CV 1/4] END depth=2, iterations=1010, learning_rate=0.011;, score=-3649.300 total time=   0.3s
[CV 2/4] END depth=2, iterations=1010, learning_rate=0.011;, score=-2234.596 total time=   0.4s
[CV 3/4] END depth=2, iterations=1010, learning_rate=0.011;, score=-3414.289 total time=   0.3s
[CV 4/4] END depth=2, iterations=1010, learning_rate=0.011;, score=-3697.542 total time=   0.3s
[CV 1/4] END depth=2, iterations=1010, learning_rate=0.012;, score=-3734.326 total time=   0.3s
[CV 2/4] END depth=2, iterations=1010, learning_rate=0.012;, score=-2240.586 total time=   0.4s
[CV 3/4] END depth=2, iterations=1010, learning_rate=0.012;, score=-3482.104 total time=   0.3s
[CV 4/4] END depth=2, iterations=1010, learning_rate=0.012;, score=-3673.208 total time=   0.4s
[CV 1/4] END depth=2, iterations=1020, learning_rate=0.008;, score=-3416.074 total time=   0.4s
[CV 2/4] END depth=2, iterations=1020, learning_rate=0.008;, score=-2036.973 total time=   0.4s
[CV 3/4] END depth=2, iterations=1020, l

[CV 3/4] END depth=3, iterations=1000, learning_rate=0.012;, score=-3493.079 total time=   0.5s
[CV 4/4] END depth=3, iterations=1000, learning_rate=0.012;, score=-3725.017 total time=   0.5s
[CV 1/4] END depth=3, iterations=1010, learning_rate=0.008;, score=-3722.629 total time=   0.6s
[CV 2/4] END depth=3, iterations=1010, learning_rate=0.008;, score=-2194.329 total time=   0.4s
[CV 3/4] END depth=3, iterations=1010, learning_rate=0.008;, score=-3234.392 total time=   0.5s
[CV 4/4] END depth=3, iterations=1010, learning_rate=0.008;, score=-3685.139 total time=   0.6s
[CV 1/4] END depth=3, iterations=1010, learning_rate=0.009;, score=-3845.072 total time=   0.5s
[CV 2/4] END depth=3, iterations=1010, learning_rate=0.009;, score=-2325.049 total time=   0.5s
[CV 3/4] END depth=3, iterations=1010, learning_rate=0.009;, score=-3250.666 total time=   0.6s
[CV 4/4] END depth=3, iterations=1010, learning_rate=0.009;, score=-3666.479 total time=   0.5s
[CV 1/4] END depth=3, iterations=1010, l

In [13]:
print( "catboost:\t\tmse = ", mse_cb, "\tr2 = ", r2_cb )
print( "grid search + catboost:\tmse = ", mse_gs, "\tr2 = ", r2_gs )
print( "linear regression:\tmse = ", mse_lr, "\tr2 = ", r2_lr )
print( "lightgbm:\t\tmse = ", mse_lgb, "\tr2 = ", r2_lgb )

catboost:		mse =  3155.5527670423476 	r2 =  0.43857241422894844
grid search + catboost:	mse =  3299.184216928826 	r2 =  0.4130178873033401
linear regression:	mse =  3131.1146628350402 	r2 =  0.4429203769660095
lightgbm:		mse =  3388.2908612250794 	r2 =  0.3971642692919719
